In [1]:
import optuna
import warnings
import gc
import os
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datetime import datetime
from glob import glob
from tqdm import tqdm

warnings.filterwarnings(action = 'ignore')

In [2]:
train_2017 = []
train_2018 = []
train_2019 = []
train_2020 = []

for i, path in enumerate(sorted(glob("Data/train/KNOW*.csv"))):
    if i == 0:
        train_2017 = pd.read_csv(path)
        pass
    elif i==1:
        train_2018 = pd.read_csv(path)
        pass
    elif i==2:
        train_2019 = pd.read_csv(path)
        pass
    else:
        train_2020 = pd.read_csv(path)
        pass
    
test_2017 = []
test_2018 = []
test_2019 = []
test_2020 = []

for i, path in enumerate(sorted(glob("Data/test/KNOW*.csv"))):
    if i == 0:
        test_2017 = pd.read_csv(path)
        pass
    elif i==1:
        test_2018 = pd.read_csv(path)
        pass
    elif i==2:
        test_2019 = pd.read_csv(path)
        pass
    else:
        test_2020 = pd.read_csv(path)
        pass

### 2018년 데이터 결측치 처리

- 중앙값으로 결측치 처리
- data leakage 고려하여 train 데이터로 결측치 처리
- cq 컬럼에는 결측치가 없음

In [3]:
for col in test_2018:
    if col == 'idx':
        continue
        
    train_2018.replace(' ', np.nan, inplace = True)
    test_2018.replace(' ', np.nan, inplace = True)

In [4]:
def modify_bq4_to_1c(data):
    for index in data[(data['bq4_1c'] == '1') | (data['bq4_1c'] == '2')].index:
        if data.loc[index, ['bq4']].isnull()[0]:
            data.loc[index, 'bq4'] = '2'
            
            data.loc[index, 'bq4_1a'] = '없음'
            data.loc[index, 'bq4_1b'] = '없음'
            data.loc[index, 'bq4_1c'] = '없음'
            pass
        else:
            
            if (data.loc[index, ['bq4_1a']].isnull()[0]) & (train_2018.loc[index, ['bq4_1b']].isnull()[0]):
                data.loc[index, 'bq4_1a'] = data.loc[index, 'bq4']
                data.loc[index, 'bq4'] = '1'
                data.loc[index, 'bq4_1c'] = np.nan
                pass
            elif data.loc[index, ['bq4_1b']].isnull()[0]:
                data.loc[index, 'bq4_1c'] = data.loc[index, 'bq4_1b']
                data.loc[index, 'bq4_1b'] = data.loc[index, 'bq4_1a']
                data.loc[index, 'bq4_1a'] = data.loc[index, 'bq4']
                data.loc[index, 'bq4'] = '1'
                pass
            else:
                data.loc[index, 'bq4_1c'] = data.loc[index, 'bq4_1b']
                data.loc[index, 'bq4_1b'] = data.loc[index, 'bq4_1a']
                data.loc[index, 'bq4_1a'] = data.loc[index, 'bq4']
                data.loc[index, 'bq4'] = '1'

modify_bq4_to_1c(train_2018)            
modify_bq4_to_1c(test_2018)

for col in ['bq4_1a', 'bq4_1b', 'bq4_1c']:
    train_2018[col].replace(np.nan, '없음', inplace = True)
    test_2018[col].replace(np.nan, '없음', inplace = True)

In [5]:
# 아스키 코드를 활용한 전처리
modify_train = []
modify_test = []

for index in train_2018.index:
    if train_2018.loc[index, ['bq5_2']].isnull()[0]:
        continue
    elif len(train_2018.loc[index, 'bq5_2']) == 1:
        if ord(train_2018.loc[index, 'bq5_2']) >= 48 and ord(train_2018.loc[index, 'bq5_2']) <= 57:
            modify_train.append(index)
    else:
        continue

for index in test_2018.index:
    if test_2018.loc[index, ['bq5_2']].isnull()[0]:
        continue
    elif len(test_2018.loc[index, 'bq5_2']) == 1:
        if ord(test_2018.loc[index, 'bq5_2']) >= 48 and ord(test_2018.loc[index, 'bq5_2']) <= 57:
            modify_test.append(index)
    else:
        continue        

In [6]:
for index in modify_train:
    if train_2018.loc[index, 'bq5'] in ['1', '2', '3', '4', '5', '6'] and train_2018.loc[index, ['bq5_1']].notnull()[0]:
        train_2018.loc[index, 'bq5_2'] = train_2018.loc[index, 'bq5_1']
        train_2018.loc[index, 'bq5_1'] = train_2018.loc[index, 'bq5']
        train_2018.loc[index, 'bq5'] = '1'
    else:
        train_2018.loc[index, 'bq5_2'] = np.nan
        train_2018.loc[index, 'bq5_1'] = np.nan
        train_2018.loc[index, 'bq5'] = '2'

In [7]:
for index in modify_test:
    if test_2018.loc[index, 'bq5'] in ['1', '2', '3', '4', '5', '6'] and test_2018.loc[index, ['bq5_1']].notnull()[0]:
        test_2018.loc[index, 'bq5_2'] = test_2018.loc[index, 'bq5_1']
        test_2018.loc[index, 'bq5_1'] = test_2018.loc[index, 'bq5']
        test_2018.loc[index, 'bq5'] = '1'
    else:
        test_2018.loc[index, 'bq5_2'] = np.nan
        test_2018.loc[index, 'bq5_1'] = np.nan
        test_2018.loc[index, 'bq5'] = '2'

In [8]:
train_2018['bq5_1'] = train_2018['bq5_1'].fillna(0)
test_2018['bq5_1'] = test_2018['bq5_1'].fillna(0)    

In [9]:
train_2018['bq5_2'] = train_2018['bq5_2'].fillna('없음')
test_2018['bq5_2'] = test_2018['bq5_2'].fillna('없음')

In [10]:
train_2018.loc[(train_2018['bq12_1'].isnull())]['bq12_1'] = '9'

In [11]:
train_indexs = train_2018[(train_2018['bq12_2'].isnull())].index
for index in train_2018[(train_2018['bq12_2'].isnull())].index:
    # 산업 유형
    industry_type = train_2018.loc[index, 'bq1']
    # 근무업체 유형
    work_company_type = train_2018.loc[index, 'bq2']
    # 교육수준
    education_level = train_2018.loc[index, 'bq7']
    # 사회적 평판_1
    social_reputation_1 = train_2018.loc[index, 'bq8_1']
    # 사회적 평판_2
    social_reputation_2 = train_2018.loc[index, 'bq8_2']
    # 직업 안정성
    job_security = train_2018.loc[index, 'bq9']
    # 근무조건
    working_conditions = train_2018.loc[index, 'bq11']
    # 발전가능성
    advancement  = train_2018.loc[index, 'bq10']
    
    if len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security) & (train_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '직업 안정성', '근무조건'
        train_2018.loc[index, 'bq12_2'] = train_2018.loc[(train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security) & (train_2018['bq11'] == working_conditions), 'bq12_2'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security)]) >= 1:
        # '산업 유형', '근무업체 유형', '직업 안정성'
        train_2018.loc[index, 'bq12_2'] = train_2018.loc[(train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq9'] == job_security), 'bq12_2'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq9'] == job_security) & (train_2018['bq10'] == advancement)]) >= 1:
        # '직업 안정성', '발전 가능성'
        train_2018.loc[index, 'bq12_2'] = train_2018[(train_2018['bq9'] == job_security) & (train_2018['bq10'] == advancement)]['bq12_2'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_2'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq9'] == job_security)]) >= 1:
        # '산업 유형', '직업 안정성'
        train_2018.loc[index, 'bq12_2'] = train_2018[(train_2018['bq1'] == industry_type) & (train_2018['bq9'] == job_security)]['bq12_2'].value_counts().index[0]
    else:
        pass

In [12]:
test_indexs = test_2018[(test_2018['bq12_2'].isnull())].index
for index in test_2018[(test_2018['bq12_2'].isnull())].index:
    # 산업 유형
    industry_type = test_2018.loc[index, 'bq1']
    # 근무업체 유형
    work_company_type = test_2018.loc[index, 'bq2']
    # 교육수준
    education_level = test_2018.loc[index, 'bq7']
    # 사회적 평판_1
    social_reputation_1 = test_2018.loc[index, 'bq8_1']
    # 사회적 평판_2
    social_reputation_2 = test_2018.loc[index, 'bq8_2']
    # 직업 안정성
    job_security = test_2018.loc[index, 'bq9']
    # 근무조건
    working_conditions = test_2018.loc[index, 'bq11']
    # 발전가능성
    advancement  = test_2018.loc[index, 'bq10']
    
    if len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security) & (test_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '직업 안정성', '근무조건'
        test_2018.loc[index, 'bq12_2'] = test_2018.loc[(test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security) & (test_2018['bq11'] == working_conditions), 'bq12_2'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security)]) >= 1:
        # '산업 유형', '근무업체 유형', '직업 안정성'
        test_2018.loc[index, 'bq12_2'] = test_2018.loc[(test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq9'] == job_security), 'bq12_2'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq9'] == job_security) & (test_2018['bq10'] == advancement)]) >= 1:
        # '직업 안정성', '발전 가능성'
        test_2018.loc[index, 'bq12_2'] = test_2018[(test_2018['bq9'] == job_security) & (test_2018['bq10'] == advancement)]['bq12_2'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_2'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq9'] == job_security)]) >= 1:
        # '산업 유형', '직업 안정성'
        test_2018.loc[index, 'bq12_2'] = test_2018[(test_2018['bq1'] == industry_type) & (test_2018['bq9'] == job_security)]['bq12_2'].value_counts().index[0]
    else:
        pass

In [13]:
train_indexs = train_2018[(train_2018['bq12_3'].isnull())].index
for index in train_2018[(train_2018['bq12_3'].isnull())].index:
    # 산업 유형
    industry_type = train_2018.loc[index, 'bq1']
    # 근무업체 유형
    work_company_type = train_2018.loc[index, 'bq2']
    # 교육수준
    education_level = train_2018.loc[index, 'bq7']
    # 사회적 평판_1
    social_reputation_1 = train_2018.loc[index, 'bq8_1']
    # 사회적 평판_2
    social_reputation_2 = train_2018.loc[index, 'bq8_2']
    # 직업 안정성
    job_security = train_2018.loc[index, 'bq9']
    # 근무조건
    working_conditions = train_2018.loc[index, 'bq11']
    # 발전가능성
    advancement  = train_2018.loc[index, 'bq10']
    
    # bq1, bq2, bq8_2, bq11
    if len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
        train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판'
        train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '근무조건'
        train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]) >= 1:
        #  '산업 유형', '근무업체 유형'
        train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]['bq12_3'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '사회적 평판'
        train_2018.loc[index, 'bq12_3'] = train_2018[(train_2018['bq12_3'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
    else:

        pass

In [14]:
test_indexs = test_2018[(test_2018['bq12_3'].isnull())].index
for index in test_2018[(test_2018['bq12_3'].isnull())].index:
    # 산업 유형
    industry_type = test_2018.loc[index, 'bq1']
    # 근무업체 유형
    work_company_type = test_2018.loc[index, 'bq2']
    # 교육수준
    education_level = test_2018.loc[index, 'bq7']
    # 사회적 평판_1
    social_reputation_1 = test_2018.loc[index, 'bq8_1']
    # 사회적 평판_2
    social_reputation_2 = test_2018.loc[index, 'bq8_2']
    # 직업 안정성
    job_security = test_2018.loc[index, 'bq9']
    # 근무조건
    working_conditions = test_2018.loc[index, 'bq11']
    # 발전가능성
    advancement  = test_2018.loc[index, 'bq10']
    
    # bq1, bq2, bq8_2, bq11
    if len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
        test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판'
        test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '근무조건'
        test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]['bq12_3'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]) >= 1:
        #  '산업 유형', '근무업체 유형'
        test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]['bq12_3'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '사회적 평판'
        test_2018.loc[index, 'bq12_3'] = test_2018[(test_2018['bq12_3'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_3'].value_counts().index[0]
    else:
        pass

In [15]:
train_indexs = train_2018[(train_2018['bq12_4'].isnull())].index
for index in train_2018[(train_2018['bq12_4'].isnull())].index:
    # 산업 유형
    industry_type = train_2018.loc[index, 'bq1']
    # 근무업체 유형
    work_company_type = train_2018.loc[index, 'bq2']
    # 교육수준
    education_level = train_2018.loc[index, 'bq7']
    # 사회적 평판_1
    social_reputation_1 = train_2018.loc[index, 'bq8_1']
    # 사회적 평판_2
    social_reputation_2 = train_2018.loc[index, 'bq8_2']
    # 직업 안정성
    job_security = train_2018.loc[index, 'bq9']
    # 근무조건
    working_conditions = train_2018.loc[index, 'bq11']
    # 발전가능성
    advancement  = train_2018.loc[index, 'bq10']
    
    
    # bq1, bq2, bq8_2, bq11
    if len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
        train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2) & (train_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판'
        train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '근무조건'
        train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type) & (train_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]) >= 1:
        #  '산업 유형', '근무업체 유형'
        train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq2'] == work_company_type)]['bq12_4'].value_counts().index[0]
    elif len(train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '사회적 평판'
        train_2018.loc[index, 'bq12_4'] = train_2018[(train_2018['bq12_4'].notnull()) & (train_2018['bq1'] == industry_type) & (train_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
    else:
        pass

In [16]:
test_indexs = test_2018[(test_2018['bq12_4'].isnull())].index
for index in test_2018[(test_2018['bq12_4'].isnull())].index:
    # 산업 유형
    industry_type = test_2018.loc[index, 'bq1']
    # 근무업체 유형
    work_company_type = test_2018.loc[index, 'bq2']
    # 교육수준
    education_level = test_2018.loc[index, 'bq7']
    # 사회적 평판_1
    social_reputation_1 = test_2018.loc[index, 'bq8_1']
    # 사회적 평판_2
    social_reputation_2 = test_2018.loc[index, 'bq8_2']
    # 직업 안정성
    job_security = test_2018.loc[index, 'bq9']
    # 근무조건
    working_conditions = test_2018.loc[index, 'bq11']
    # 발전가능성
    advancement  = test_2018.loc[index, 'bq10']
    

    # bq1, bq2, bq8_2, bq11
    if len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판', '근무조건'
        test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2) & (test_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '근무업체 유형', '사회적 평판'
        test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]) >= 1:
        # '산업 유형', '근무업체 유형', '근무조건'
        test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type) & (test_2018['bq11'] == working_conditions)]['bq12_4'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]) >= 1:
        #  '산업 유형', '근무업체 유형'
        test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq2'] == work_company_type)]['bq12_4'].value_counts().index[0]
    elif len(test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]) >= 1:
        # '산업 유형', '사회적 평판'
        test_2018.loc[index, 'bq12_4'] = test_2018[(test_2018['bq12_4'].notnull()) & (test_2018['bq1'] == industry_type) & (test_2018['bq8_2'] == social_reputation_2)]['bq12_4'].value_counts().index[0]
    else:
        pass

In [17]:
train_2018['bq223'].replace('3.0', '3', inplace = True)
train_2018['bq223'].replace(3.0, '3', inplace = True)

test_2018['bq223'].replace('3.0', '3', inplace = True)
test_2018['bq223'].replace(3.0, '3', inplace = True)

train_2018.loc[(train_2018['bq223'] != '3'), 'bq223'] = np.nan
test_2018.loc[(test_2018['bq223'] != '3'), 'bq223'] = np.nan

train_2018.loc[(train_2018['bq223'] == '3'), 'bq221'] = np.nan
train_2018.loc[(train_2018['bq223'] == '3'), 'bq222'] = np.nan

test_2018.loc[(test_2018['bq223'] == '3'), 'bq221'] = np.nan
test_2018.loc[(test_2018['bq223'] == '3'), 'bq222'] = np.nan

train_2018['bq221'].replace('2', np.nan, inplace = True)
train_2018['bq221'].replace('3', np.nan, inplace = True)
test_2018['bq221'].replace('2', np.nan, inplace = True)
test_2018['bq221'].replace('3', np.nan, inplace = True)

train_2018['bq222'].replace('0', np.nan, inplace = True)
test_2018['bq222'].replace('0', np.nan, inplace = True)
test_2018['bq222'].replace('1', np.nan, inplace = True)
test_2018['bq222'].replace('3', np.nan, inplace = True)

for col in ['bq221', 'bq222', 'bq223']:
    train_2018[col].replace(np.nan, '0', inplace = True)
    test_2018[col].replace(np.nan, '0', inplace = True)    

In [18]:
train_2018.loc[(train_2018['bq231'] != '1'), 'bq231'] = np.nan
test_2018.loc[(test_2018['bq231'] != '1'), 'bq231'] = np.nan

train_2018.loc[(train_2018['bq232'] != 2), 'bq232'] = np.nan
test_2018.loc[(test_2018['bq232'] != 2), 'bq232'] = np.nan

train_2018.loc[(train_2018['bq233'] != 3), 'bq233'] = np.nan
test_2018.loc[(test_2018['bq233'] != 3), 'bq233'] = np.nan

train_2018['bq234'].replace('4.0', '4', inplace = True)
test_2018['bq234'].replace('4.0', '4', inplace = True)
train_2018.loc[(train_2018['bq234'] != '4'), 'bq234'] = np.nan
test_2018.loc[(test_2018['bq234'] != '4'), 'bq234'] = np.nan

train_2018.loc[(train_2018['bq235'] != '5'), 'bq235'] = np.nan
test_2018.loc[(test_2018['bq235'] != '5'), 'bq235'] = np.nan

train_2018[['bq231', 'bq232', 'bq233', 'bq234', 'bq235']] = train_2018[['bq231', 'bq232', 'bq233', 'bq234', 'bq235']].fillna(0)
test_2018[['bq231', 'bq232', 'bq233', 'bq234', 'bq235']] = test_2018[['bq231', 'bq232', 'bq233', 'bq234', 'bq235']].fillna(0)

In [19]:
train_2018.loc[(train_2018['bq241'] != '1'), 'bq241'] = np.nan
test_2018.loc[(test_2018['bq241'] != '1'), 'bq241'] = np.nan

train_2018.loc[(train_2018['bq242'] != '2'), 'bq242'] = np.nan
test_2018.loc[(test_2018['bq242'] != '2'), 'bq242'] = np.nan

train_2018.loc[(train_2018['bq243'] != '3'), 'bq243'] = np.nan
test_2018.loc[(test_2018['bq243'] != '3'), 'bq243'] = np.nan

train_2018.loc[(train_2018['bq244'] != '4'), 'bq244'] = np.nan
test_2018.loc[(test_2018['bq244'] != '4'), 'bq244'] = np.nan

train_2018.loc[(train_2018['bq245'] != '5'), 'bq245'] = np.nan
test_2018.loc[(test_2018['bq245'] != '5'), 'bq245'] = np.nan

train_2018[['bq241', 'bq242', 'bq243', 'bq244', 'bq245']] = train_2018[['bq241', 'bq242', 'bq243', 'bq244', 'bq245']].fillna(0)
test_2018[['bq241', 'bq242', 'bq243', 'bq244', 'bq245']] = test_2018[['bq241', 'bq242', 'bq243', 'bq244', 'bq245']].fillna(0)

In [20]:
train_2018['bq25'].replace(1, '1', inplace = True)
train_2018['bq25'].replace(2, '2', inplace = True)
train_2018['bq25'].replace(3, '3', inplace = True)

test_2018['bq25'].replace(1.0, '1', inplace = True)
test_2018['bq25'].replace(2.0, '2', inplace = True)
test_2018['bq25'].replace(3.0, '3', inplace = True)

train_2018.loc[(train_2018['bq25'].isnull()), 'bq25'] = '1'
test_2018.loc[(test_2018['bq25'].isnull()), 'bq25'] = '1'

train_2018.loc[(train_2018['bq25'] == '1') & (train_2018['bq25_1'].isnull()), 'bq25_1'] = '2'
test_2018.loc[(test_2018['bq25'] == '1') & (test_2018['bq25_1'].isnull()), 'bq25_1'] = '2'

train_2018['bq25_1'].replace('3', '2', inplace = True)
test_2018['bq25_1'].replace('3', '2', inplace = True)

train_2018['bq25_1'].replace(np.nan, 0, inplace = True)
test_2018['bq25_1'].replace(np.nan, 0, inplace = True)

In [21]:
train_2018.replace('1.0', '1', inplace = True)
train_2018.replace(1.0, '1', inplace = True)

test_2018.replace('1.0', '1', inplace = True)
test_2018.replace(1.0, '1', inplace = True)

train_2018.replace('2.0', '2', inplace = True)
train_2018.replace(2.0, '2', inplace = True)

test_2018.replace('2.0', '2', inplace = True)
test_2018.replace(2.0, '2', inplace = True)

train_2018.replace('3.0', '3', inplace = True)
train_2018.replace(3.0, '3', inplace = True)

test_2018.replace('3.0', '3', inplace = True)
test_2018.replace(3.0, '3', inplace = True)

train_2018.loc[(train_2018['bq26_1'].isnull()), 'bq26_1'] = '1'
test_2018.loc[(test_2018['bq26_1'].isnull()), 'bq26_1'] = '1'

In [22]:
train_2018.loc[(train_2018['bq28_1'].isnull()), 'bq28_1'] = '없음'
test_2018.loc[(test_2018['bq28_1'].isnull()), 'bq28_1'] = '없음'

train_2018.loc[(train_2018['bq29'].isnull()), 'bq29'] = '없음'
test_2018.loc[(test_2018['bq29'].isnull()), 'bq29'] = '없음'

train_2018.loc[(train_2018['bq31'].isnull()), 'bq31'] = '없음'
test_2018.loc[(test_2018['bq31'].isnull()), 'bq31'] = '없음'

train_2018.loc[(train_2018['bq32'].isnull()), 'bq32'] = '없음'
test_2018.loc[(test_2018['bq32'].isnull()), 'bq32'] = '없음'

train_2018.loc[(train_2018['bq33'].isnull()), 'bq33'] = '없음'
test_2018.loc[(test_2018['bq33'].isnull()), 'bq33'] = '없음'

train_2018.loc[(train_2018['bq36'] == '디자이너'), 'bq36'] = train_2018.loc[(train_2018['bq36'].notnull() & (train_2018['bq36'] != '디자이너'))]['bq36'].astype('int').quantile(q = 0.5)
train_2018.loc[(train_2018['bq36'].isnull()), 'bq36'] = train_2018.loc[(train_2018['bq36'].notnull() & (train_2018['bq36'] != '디자이너'))]['bq36'].astype('int').quantile(q = 0.5)
train_2018['bq36'].replace('2', '20', inplace = True)
train_2018['bq36'].replace('3', '30', inplace = True)
train_2018['bq36'].replace('4', '40', inplace = True)
train_2018['bq36'].replace(5.0, '50', inplace = True)

train_2018.loc[(train_2018['bq37'] == '1'), 'bq37_1'] = '없음'
test_2018.loc[(test_2018['bq37'] == '1'), 'bq37_1'] = '없음'

In [23]:
haha = ['1', '2', '3', '4', '5', '6']
train_indexs = []
for index in train_2018.index:
    value = train_2018.loc[index, 'bq37']
    
    if value not in haha:
        train_indexs.append(index)
        if value != '공고졸업':
            train_2018.loc[index, 'bq37'] = '4'
            train_2018.loc[index, 'bq37_1'] = value
        else:
            train_2018.loc[index, 'bq37'] = '2'
            train_2018.loc[index, 'bq37_1'] = value
            
for index in test_2018.index:
    value = test_2018.loc[index, 'bq37']
    
    if value not in haha:
        if value != '공고졸업':
            test_2018.loc[index, 'bq37'] = '4'
            test_2018.loc[index, 'bq37_1'] = value
        else:
            test_2018.loc[index, 'bq37'] = '2'
            test_2018.loc[index, 'bq37_1'] = value            

In [24]:
train_2018['bq38_1'] = train_2018['bq38_1'].fillna(0)
train_2018['bq38_1'] = train_2018['bq38_1'].replace('5', 0)

test_2018['bq38_1'] = test_2018['bq38_1'].fillna(0)
test_2018['bq38_1'] = test_2018['bq38_1'].replace('70', 0)

In [25]:
train_2018['bq38_2'] = train_2018['bq38_2'].fillna(0)
train_2018['bq38_2'] = train_2018['bq38_2'].replace('48', 0)
train_2018['bq38_2'] = train_2018['bq38_2'].replace('1', 0)

test_2018['bq38_2'] = test_2018['bq38_2'].fillna(0)
test_2018['bq38_2'] = test_2018['bq38_2'].replace('2', 0)
test_2018['bq38_2'] = test_2018['bq38_2'].replace('1', 0)

In [26]:
train_2018['bq41_1'] = train_2018['bq41_1'].fillna(0)
test_2018['bq41_1'] = test_2018['bq41_1'].fillna(0)

train_2018['bq41_2'] = train_2018['bq41_2'].fillna(0)
test_2018['bq41_2'] = test_2018['bq41_2'].fillna(0)

train_2018['bq41_3'] = train_2018['bq41_3'].fillna(0)
test_2018['bq41_3'] = test_2018['bq41_3'].fillna(0)

train_2018['bq39'] = train_2018['bq39'].fillna(0)

test_2018['bq39'] = test_2018['bq39'].fillna(0)

In [27]:
for index in train_2018['bq39'].loc[train_2018['bq39'].astype(int) > 2].index:
    train_2018['bq39'] = train_2018['bq39'].replace(train_2018['bq39'][index], 1)

for index in test_2018['bq39'].loc[test_2018['bq39'].astype(int) > 2].index:
    test_2018['bq39'] = test_2018['bq39'].replace(test_2018['bq39'][index], 1)

In [28]:
train_2018['bq37_1'] = train_2018['bq37_1'].fillna('없음')
test_2018['bq37_1'] = test_2018['bq37_1'].fillna('없음')

train_2018['bq30'] = train_2018['bq30'].fillna('없음')
test_2018['bq30'] = test_2018['bq30'].fillna('없음')

In [29]:
train_2018.loc[train_2018['bq40'].str.contains('[가-힣]', na = False), ['bq40']] = np.nan

for index in train_2018.loc[train_2018['bq40'].astype(float) > 100].index:
    train_2018['bq40'] = train_2018['bq40'].replace(train_2018['bq40'][index], int(train_2018['bq40'].mode()))

for index in test_2018.loc[test_2018['bq40'].astype(float) > 100].index:
    test_2018['bq40'] = test_2018['bq40'].replace(test_2018['bq40'][index], int(train_2018['bq40'].mode()))

train_2018['bq40'] = train_2018['bq40'].fillna(int(train_2018['bq40'].mode()))
test_2018['bq40'] = test_2018['bq40'].fillna(int(train_2018['bq40'].mode()))

In [30]:
train_2018['bq21'] = train_2018['bq21'].fillna(int(train_2018['bq21'].mode()))
train_2018['bq12_1'] = train_2018['bq12_1'].fillna(int(train_2018['bq12_1'].mode()))
train_2018['bq38'] = train_2018['bq38'].fillna(int(train_2018['bq38'].mode()))

test_2018['bq38'] = test_2018['bq38'].fillna(int(train_2018['bq38'].mode()))

In [31]:
train_2018['bq26_1a'] = train_2018['bq26_1a'].fillna(0)
train_2018['bq26_2a'] = train_2018['bq26_2a'].fillna(0)
train_2018['bq26_3a'] = train_2018['bq26_3a'].fillna(0)
train_2018['bq26_4a'] = train_2018['bq26_4a'].fillna(0)

In [32]:
test_2018['bq26_1a'] = test_2018['bq26_1a'].fillna(0)
test_2018['bq26_2a'] = test_2018['bq26_2a'].fillna(0)
test_2018['bq26_3a'] = test_2018['bq26_3a'].fillna(0)
test_2018['bq26_4a'] = test_2018['bq26_4a'].fillna(0)

In [33]:
train_2018['bq26_2'] = train_2018['bq26_2'].fillna(int(train_2018['bq26_2'].mode()))
test_2018['bq26_2'] = test_2018['bq26_2'].fillna(int(train_2018['bq26_2'].mode()))

In [34]:
train_2018['bq26_3'] = train_2018['bq26_3'].fillna(int(train_2018['bq26_3'].mode()))
test_2018['bq26_3'] = test_2018['bq26_3'].fillna(int(train_2018['bq26_3'].mode()))

In [35]:
train_2018['bq26_4'] = train_2018['bq26_4'].fillna(int(train_2018['bq26_4'].mode()))
test_2018['bq26_4'] = test_2018['bq26_4'].fillna(int(train_2018['bq26_4'].mode()))

In [36]:
train_2018.loc[train_2018['bq34'].str.contains('[가-힣]', na = False), ['bq34']] = int(train_2018['bq34'].mode())

In [37]:
train_2018.loc[train_2018['bq28'].str.contains('[가-힣]', na = False), ['bq28']] = int(train_2018['bq28'].mode())
test_2018.loc[test_2018['bq28'].str.contains('[가-힣]', na = False), ['bq28']] = int(train_2018['bq28'].mode())

In [38]:
for n, x in enumerate(train_2018['bq35']) : 
    if train_2018['bq35'][n] == '1':
        pass
    elif train_2018['bq35'][n] == '2':
        pass
    else :
        train_2018['bq35'][n] = int(train_2018['bq35'].mode())

In [39]:
for n, x in enumerate(test_2018['bq35']) : 
    if test_2018['bq35'][n] == '1':
        pass
    elif test_2018['bq35'][n] == '2':
        pass
    else :
        test_2018['bq35'][n] = int(test_2018['bq35'].mode())

### 2018년 데이터 object 컬럼 처리 

In [40]:
convert = []
for col in train_2018.select_dtypes('object'):
    if train_2018[col].nunique() < 20:
        train_2018[col] = train_2018[col].astype(float)

for col in ['bq1', 'bq21', 'bq36', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3']:
    train_2018[col] = train_2018[col].astype(float)

In [41]:
convert = []
for col in test_2018.select_dtypes('object'):
    if test_2018[col].nunique() < 20:
        test_2018[col] = test_2018[col].astype(float)

for col in ['bq1', 'bq21', 'bq36', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3']:
    test_2018[col] = test_2018[col].astype(float)

In [46]:
train_2018.to_csv('Data/train/1.preprocessing_train_2018.csv', index = False)
test_2018.to_csv('Data/test/1.preprocessing_test_2018.csv', index = False)